In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
import tensorflow as tf
import pickle


def load_and_preprocess_data(file_path):
    df = pd.read_csv(file_path)
    df['datetime'] = pd.to_datetime(df['datetime'])

    # Initialize scaler
    scaler = MinMaxScaler(feature_range=(0, 1))
    df[['temp', 'humidity', 'sealevelpressure']] = scaler.fit_transform(
        df[['temp', 'humidity', 'sealevelpressure']]
    )

    return df, scaler


def create_time_slices(df):
    time_slices = {
        'morning': df[df['datetime'].dt.hour.between(6, 11)],
        'afternoon': df[df['datetime'].dt.hour.between(12, 17)],
        'evening': df[df['datetime'].dt.hour.between(18, 23)],
        'night': df[df['datetime'].dt.hour.between(0, 5)]
    }
    return time_slices


def create_sequences(data, n_steps, forecast_horizon):
    X, y, dates = [], [], []
    for i in range(len(data) - n_steps - forecast_horizon + 1):
        X.append(data.iloc[i:i + n_steps][['temp', 'humidity', 'sealevelpressure']].values)
        y.append(data.iloc[i + n_steps:i + n_steps + forecast_horizon][['temp', 'humidity', 'sealevelpressure']].values.reshape(-1))
        dates.append(data.iloc[i + n_steps:i + n_steps + forecast_horizon]['datetime'].values)
    return np.array(X), np.array(y), np.array(dates)


def create_qos_models(input_shape):
    models = {}

    # High accuracy model
    models['high_accuracy'] = Sequential([
        LSTM(128, return_sequences=True, input_shape=input_shape),
        LSTM(64, return_sequences=True),
        LSTM(32),
        Dense(18)
    ])

    # Medium priority model
    models['medium_priority'] = Sequential([
        LSTM(64, return_sequences=True, input_shape=input_shape),
        LSTM(32),
        Dense(18)
    ])

    # Low latency model
    models['low_latency'] = Sequential([
        LSTM(32, input_shape=input_shape),
        Dense(18)
    ])

    for model in models.values():
        model.compile(optimizer='adam', loss='mean_squared_error')

    return models



def create_feature_models(input_shape):
    models = {}

    features = ['temp', 'humidity', 'sealevelpressure']
    for feature in features:
        models[feature] = Sequential([
            LSTM(50, return_sequences=True, input_shape=(input_shape[0], 1)),
            LSTM(50),
            Dense(6)
        ])
        models[feature].compile(optimizer='adam', loss='mean_squared_error')

    return models


def create_network_slices():
    """
    Define 5G network slices corresponding to QoS requirements.
    """
    slices = {
        'eMBB': 'high_accuracy',  # High data rate, less focus on latency
        'URLLC': 'low_latency',   # Ultra-low latency, high reliability
        'mMTC': 'medium_priority'  # Optimized for massive connections
    }
    return slices


def train_models_with_slices(file_path, save_path='models/'):
    # Load and preprocess data
    print("Loading and preprocessing data...")
    df, scaler = load_and_preprocess_data(file_path)

    # Save scaler for later use
    with open(f'{save_path}scaler.pkl', 'wb') as f:
        pickle.dump(scaler, f)

    # Create time slices
    print("Creating time slices...")
    time_slices = create_time_slices(df)

    # Parameters
    n_steps = 24
    forecast_horizon = 6
    input_shape = (n_steps, 3)

    # Initialize models
    print("Initializing models...")
    qos_models = create_qos_models(input_shape)
    feature_models = create_feature_models(input_shape)

    # Define 5G network slices
    network_slices = create_network_slices()

    # Train models for each time slice
    trained_models = {'qos': {}, 'feature': {}, 'network_slices': {}}

    for time_slice_name, time_slice_data in time_slices.items():
        print(f"\nTraining models for {time_slice_name} slice...")

        # Create sequences
        X, y, dates = create_sequences(time_slice_data, n_steps, forecast_horizon)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Train QoS models for each network slice
        for slice_name, qos_level in network_slices.items():
            print(f"Training {slice_name} (mapped to {qos_level}) QoS model...")
            model = qos_models[qos_level]

            # Customize training parameters based on slice
            if slice_name == 'eMBB':
                epochs, batch_size = 15, 64  # More epochs, larger batch for high data rate
            elif slice_name == 'URLLC':
                epochs, batch_size = 5, 16   # Fewer epochs, smaller batch for low latency
            else:  # mMTC
                epochs, batch_size = 10, 32  # Balanced training for massive connections

            model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                      validation_data=(X_test, y_test), verbose=1)
            trained_models['network_slices'][f'{time_slice_name}_{slice_name}'] = model

            # Save the model
            save_model(model, f'{save_path}{time_slice_name}_{slice_name}_model.keras')

        # Train feature-specific models
        for feature_name, model in feature_models.items():
            print(f"Training {feature_name} feature model...")
            feature_idx = ['temp', 'humidity', 'sealevelpressure'].index(feature_name)
            X_feature = X[:, :, feature_idx:feature_idx + 1]
            y_feature = y[:, feature_idx::3]

            X_train_feature, X_test_feature, y_train_feature, y_test_feature = train_test_split(
                X_feature, y_feature, test_size=0.2, random_state=42
            )

            model.fit(X_train_feature, y_train_feature, epochs=10, batch_size=32,
                      validation_data=(X_test_feature, y_test_feature), verbose=1)
            trained_models['feature'][f'{time_slice_name}_{feature_name}'] = model

            # Save the model
            save_model(model, f'{save_path}{time_slice_name}_{feature_name}_model.keras')

    print("\nTraining completed with 5G network slicing. Models saved.")
    return trained_models, scaler


if __name__ == "__main__":
    # Set file paths
    file_path = '/content/drive/My Drive/Weather/Filtered_Weather_Data.csv'
    save_path = '/content/drive/My Drive/Weather/models/'

    # Train models with 5G slicing
    trained_models, scaler = train_models_with_slices(file_path, save_path)


Loading and preprocessing data...
Creating time slices...
Initializing models...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



Training models for morning slice...
Training eMBB (mapped to high_accuracy) QoS model...
Epoch 1/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 289ms/step - loss: 0.2639 - val_loss: 0.1977
Epoch 2/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.1886 - val_loss: 0.1310
Epoch 3/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.1187 - val_loss: 0.0672
Epoch 4/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 0.0642 - val_loss: 0.0400
Epoch 5/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 0.0393 - val_loss: 0.0320
Epoch 6/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.0322 - val_loss: 0.0300
Epoch 7/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 0.0309 - val_loss: 0.0301
Epoch 8/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 0.0305 - val_loss: 0.0299
Epoch 9/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.0297 - val_loss: 0.0288
Epoch 10/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 0.0294 - val_loss: 0.0290
Epoch 11/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 0.0281 - val_loss: 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
import pickle


def load_and_preprocess_data(file_path):
    df = pd.read_csv(file_path)
    df['datetime'] = pd.to_datetime(df['datetime'])

    # Initialize scaler
    scaler = MinMaxScaler(feature_range=(0, 1))
    df[['temp', 'humidity', 'sealevelpressure']] = scaler.fit_transform(
        df[['temp', 'humidity', 'sealevelpressure']]
    )

    return df, scaler


def create_time_slices(df):
    time_slices = {
        'morning': df[df['datetime'].dt.hour.between(6, 11)],
        'afternoon': df[df['datetime'].dt.hour.between(12, 17)],
        'evening': df[df['datetime'].dt.hour.between(18, 23)],
        'night': df[df['datetime'].dt.hour.between(0, 5)]
    }
    return time_slices


def create_sequences(data, n_steps, forecast_horizon):
    X, y, dates = [], [], []
    for i in range(len(data) - n_steps - forecast_horizon + 1):
        X.append(data.iloc[i:i + n_steps][['temp', 'humidity', 'sealevelpressure']].values)
        y.append(data.iloc[i + n_steps:i + n_steps + forecast_horizon][['temp', 'humidity', 'sealevelpressure']].values.reshape(-1))
        dates.append(data.iloc[i + n_steps:i + n_steps + forecast_horizon]['datetime'].values)
    return np.array(X), np.array(y), np.array(dates)


def create_qos_models(input_shape, forecast_horizon):
    """
    Create QoS models for different priorities.
    Args:
        input_shape (tuple): Shape of the input data (timesteps, features).
        forecast_horizon (int): Number of future steps to predict.

    Returns:
        dict: Dictionary of models for different QoS levels.
    """
    output_units = forecast_horizon * input_shape[1]  # forecast_horizon * number of features
    models = {}

    # High accuracy model
    models['high_accuracy'] = Sequential([
        LSTM(128, return_sequences=True, input_shape=input_shape),
        LSTM(64, return_sequences=True),
        LSTM(32),
        Dense(output_units)  # Adjusted to match y's shape
    ])

    # Medium priority model
    models['medium_priority'] = Sequential([
        LSTM(64, return_sequences=True, input_shape=input_shape),
        LSTM(32),
        Dense(output_units)
    ])

    # Low latency model
    models['low_latency'] = Sequential([
        LSTM(32, input_shape=input_shape),
        Dense(output_units)
    ])

    # Compile all models
    for model in models.values():
        model.compile(optimizer='adam', loss='mean_squared_error')

    return models


def create_network_slices():
    """
    Define 5G network slices corresponding to QoS requirements.
    """
    slices = {
        'eMBB': 'high_accuracy',  # High data rate, less focus on latency
        'URLLC': 'low_latency',   # Ultra-low latency, high reliability
        'mMTC': 'medium_priority'  # Optimized for massive connections
    }
    return slices


def train_models_with_slices(file_path, save_path='models/'):
    # Load and preprocess data
    print("Loading and preprocessing data...")
    df, scaler = load_and_preprocess_data(file_path)

    # Save scaler for later use
    with open(f'{save_path}scaler.pkl', 'wb') as f:
        pickle.dump(scaler, f)

    # Create time slices
    print("Creating time slices...")
    time_slices = create_time_slices(df)

    # Parameters
    n_steps = 24
    forecast_horizon = 6
    input_shape = (n_steps, 3)

    # Initialize models
    print("Initializing models...")
    qos_models = create_qos_models(input_shape, forecast_horizon)

    # Define 5G network slices
    network_slices = create_network_slices()

    # Train models for each time slice
    trained_models = {'network_slices': {}}

    for time_slice_name, time_slice_data in time_slices.items():
        print(f"\nTraining models for {time_slice_name} slice...")

        # Create sequences
        X, y, dates = create_sequences(time_slice_data, n_steps, forecast_horizon)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Train QoS models for each network slice
        for slice_name, qos_level in network_slices.items():
            print(f"Training {slice_name} (mapped to {qos_level}) QoS model...")
            model = qos_models[qos_level]

            # Customize training parameters based on slice
            if slice_name == 'eMBB':
                epochs, batch_size = 15, 64  # More epochs, larger batch for high data rate
            elif slice_name == 'URLLC':
                epochs, batch_size = 5, 16   # Fewer epochs, smaller batch for low latency
            else:  # mMTC
                epochs, batch_size = 10, 32  # Balanced training for massive connections

            model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                      validation_data=(X_test, y_test), verbose=1)
            trained_models['network_slices'][f'{time_slice_name}_{slice_name}'] = model

            # Save the model
            save_model(model, f'{save_path}{time_slice_name}_{slice_name}_model.keras')

    print("\nTraining completed with 5G network slicing. Models saved.")
    return trained_models, scaler


if __name__ == "__main__":
    # Set file paths
    file_path = '/content/drive/My Drive/Weather/Filtered_Weather_Data.csv'
    save_path = '/content/drive/My Drive/Weather/models/'

    # Train models with 5G slicing
    trained_models, scaler = train_models_with_slices(file_path, save_path)


Loading and preprocessing data...
Creating time slices...
Initializing models...

Training models for morning slice...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Training eMBB (mapped to high_accuracy) QoS model...
Epoch 1/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 294ms/step - loss: 0.2481 - val_loss: 0.1748
Epoch 2/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - loss: 0.1608 - val_loss: 0.1033
Epoch 3/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 0.0935 - val_loss: 0.0818
Epoch 4/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 0.0702 - val_loss: 0.0523
Epoch 5/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - loss: 0.0468 - val_loss: 0.0349
Epoch 6/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 0.0364 - val_loss: 0.0286
Epoch 7/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 0.0314 - val_loss: 0.0278
Epoch 8/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 0.0296 - val_loss: 0.0262
Epoch 9/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 0.0285 - val_loss: 0.0269
Epoch 10/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.0286 - val_loss: 0.0273
Epoch 11/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - loss: 0.0284 - val_loss: 0.0273
Epoch 12/15
3/3 ━━━━━━━━━━━━━━━

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import tensorflow as tf


def create_model(input_shape=(24, 3)):
    """
    Create an LSTM-based model.
    """
    model = Sequential([
        LSTM(64, activation='tanh', return_sequences=True, input_shape=input_shape),
        LSTM(32, activation='tanh', return_sequences=False),
        Dense(72)  # Output size: 72 = 24 steps * 3 features (temp, humidity, pressure)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model


# Save the model in the new `.keras` format
def save_model(model, models_path, time_slice, qos_level):
    model_name = f'{time_slice}_{qos_level}_model.keras'
    model.save(f'{models_path}{model_name}', save_format='keras')
    print(f"Model saved as {model_name}")


# Example Usage
if __name__ == "__main__":
    models_path = '/content/drive/My Drive/Weather/models/'
    time_slice = 'evening'
    qos_level = 'low_latency'

    # Define the input shape
    input_shape = (24, 3)

    # Create and save the model
    model = create_model(input_shape=input_shape)
    save_model(model, models_path, time_slice, qos_level)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model saved as evening_low_latency_model.keras


In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
import pickle


def load_single_model(models_path, time_slice, qos_level=None, feature=None):
    """
    Load a single model based on the specified time slice and type.
    """
    if qos_level:
        model_name = f'{time_slice}_{qos_level}_model.keras'
    elif feature:
        model_name = f'{time_slice}_{feature}_model.keras'
    else:
        raise ValueError("Either qos_level or feature must be specified.")

    print(f"Loading model: {model_name}")
    return load_model(f'{models_path}{model_name}', compile=True)


def preprocess_data(data_path, scaler, n_steps=24):
    """
    Preprocess data: load, scale, and prepare it for prediction.
    """
    print("Loading and preprocessing data...")
    df = pd.read_csv(data_path)
    print(df)
    df['datetime'] = pd.to_datetime(df['date'])  # Ensure datetime column is correct

    # Scale relevant features
    print("Scaling data...")
    df[['temp', 'humidity', 'sealevelpressure']] = scaler.transform(
        df[['temp', 'humidity', 'sealevelpressure']]
    )

    # Select the last n_steps
    if len(df) < n_steps:
        raise ValueError(f"Not enough data for {n_steps} steps. Data length: {len(df)}")
    input_data = df.iloc[-n_steps:][['temp', 'humidity', 'sealevelpressure']].values.reshape(1, n_steps, 3)

    return input_data, df['datetime'].iloc[-1]


def make_single_prediction(input_data, model, scaler):
    """
    Make a single prediction and inverse scale the results.
    """
    print("Making prediction...")
    prediction = model.predict(input_data)
    print(f"Raw prediction shape: {prediction.shape}")

    # Reshape prediction to match the scaler's expected input (n_samples, 3)
    prediction = prediction.reshape(-1, 3)

    print("Prediction made. Inversing scaling...")
    inversed_prediction = scaler.inverse_transform(prediction)
    return inversed_prediction


def main_single_prediction(data_path, models_path):
    """
    Main function for a single prediction with network slicing.
    """
    print("Loading scaler...")
    with open(f'{models_path}scaler.pkl', 'rb') as f:
        scaler = pickle.load(f)

    # Determine time slice based on current time (example: morning)
    time_slice = 'evening'  # Replace with logic to determine based on datetime if needed
    qos_level = 'low_latency'  # Example, can be dynamically set based on network conditions

    print(f"Selected time slice: {time_slice}, QoS level: {qos_level}")

    # Load model for the selected QoS level and time slice
    model = load_single_model(models_path, time_slice, qos_level=qos_level)

    # Preprocess data
    input_data, last_datetime = preprocess_data(data_path, scaler)

    # Predict
    predictions = make_single_prediction(input_data, model, scaler)

    # Display results
    print(f"\nPrediction for {time_slice} at {last_datetime}:")
    temperature_f = predictions[0][0]
    temperature_c = (temperature_f - 32) * 5 / 9
    print(f"Temperature: {temperature_c:.2f}°C")

    print(f"Humidity: {predictions[0][1]:.2f}%")
    print(f"Pressure: {predictions[0][2]:.2f} hPa")


if __name__ == "__main__":
    # Set file paths
    data_path = '/content/drive/My Drive/Weather/filtered_data.csv'
    models_path = '/content/drive/My Drive/Weather/models/'

    # Run single prediction
    main_single_prediction(data_path, models_path)


Loading scaler...
Selected time slice: evening, QoS level: low_latency
Loading model: evening_low_latency_model.keras
Loading and preprocessing data...
                   date  temp  humidity  sealevelpressure
0   2024-11-30T00:00:00    20     45.90              1014
1   2024-11-30T01:00:00    19     48.84              1014
2   2024-11-30T02:00:00    17     55.39              1013
3   2024-11-30T03:00:00    18     48.57              1013
4   2024-11-30T04:00:00    17     51.73              1013
5   2024-11-30T05:00:00    16     55.13              1013
6   2024-11-30T06:00:00    17     51.73              1013
7   2024-11-30T07:00:00    16     59.03              1014
8   2024-11-30T08:00:00    17     51.73              1015
9   2024-11-30T09:00:00    20     45.90              1016
10  2024-11-30T10:00:00    22     40.59              1016
11  2024-11-30T11:00:00    24     35.96              1015
12  2024-11-30T12:00:00    25     29.53              1014
13  2024-11-30T13:00:00    26     25

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 18 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 281ms/step
Raw prediction shape: (1, 72)
Prediction made. Inversing scaling...

Prediction for evening at 2024-11-30 23:00:00:
Temperature: 21.65°C
Humidity: 12.04%
Pressure: 1000.67 hPa
